In [72]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
from collections import defaultdict
from itertools import combinations
import networkx as nx
import community
from utils.cluster import cluster
from utils.recombinations import recombinations
from matplotlib.colors import LogNorm, LinearSegmentedColormap
from pandas.io.formats.style import Styler
from utils.preprocessing import preprocessing
import os
import seaborn as sns
import igraph as ig
import scipy.stats as stats
import mplcursors
import plotly.express as px
import plotly.graph_objs as go
import plotly.io as pio
from Bio import pairwise2
import difflib

sjogren_path = '../Data/new_sjogren_file/'
healthy_path = '../Data/in_house_healthy/'
sjogren_clone_path = '../Data/IgPhyML_input/sjogren/'

/Users/daeheonkwon/anaconda3/envs/IP/lib/python3.10/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [8]:
for file in os.listdir(sjogren_path):
    df = preprocessing(pd.read_csv(os.path.join(sjogren_path, file), sep='\t'))
    df()['locus'] = 'IGH'
    df()[df()['v_call'] == 'IGHV1-69'].to_csv(os.path.join('../Data/IgPhyML_input/sjogren/', file), sep='\t', index=False, header=True)

In [0]:
df.get_Vgene('IGHV1-69').to_csv(os.path.join('../Data/IgPhyML_input/sjogren/', 'S8_add_d_gene_IGHV1-69.tsv'), sep='\t', index=False, header=True)

In [95]:
print(df.iloc[29]['sequence_alignment'])
print(df.iloc[29]['sequence'])

GCTGCTGGAGGCACCTTCAGCACCTATACTATCAGGTGGGTGCGACAGGCCCCTGAACAAGGGCTTGAGTGGATGGGAGAGATCGTCCCTATCTTTGGAAAAGTAAACTACGCACAGAAGTTCCAGGGCAGAGTCACGATTACCG------------------CAGCCGACATGGAACTGAGCAGCCTGAGATCTGAGGACACGGCCGTGTATTACTGTGCGATGACCTTAAAATTTGATCCGGTATTTGAATATTGGGGTCAGGGAACCCTGGTCACCGTCTCCTCA
GCTGCTGGAGGCACCTTCAGCACCTATACTATCAGGTGGGTGCGACAGGCCCCTGAACAAGGGCTTGAGTGGATGGGAGAGATCGTCCCTATCTTTGGAAAAGTAAACTACGCACAGAAGTTCCAGGGCAGAGTCACGATTACCGCAGCCGACATGGAACTGAGCAGCCTGAGATCTGAGGACACGGCCGTGTATTACTGTGCGATGACCTTAAAATTTGATCCGGTATTTGAATATTGGGGTCAGGGAACCCTGGTCACCGTCTCCTCA


In [98]:
df = pd.read_csv(os.path.join(sjogren_clone_path, 'S8_add_d_gene_clone-pass.tsv'), sep='\t')

# Function to find the first and last aligned positions for a single row
def find_first_last_aligned_positions(row):
    sequence = row['sequence']
    sequence_alignment = row['sequence_alignment']

    # Initialize variables to track positions
    first_position = -1
    last_position = -1

    # Iterate through the sequence and sequence_alignment
    i = 0  # Index for the sequence
    j = 0  # Index for the sequence_alignment
    while i < len(sequence) and j < len(sequence_alignment):
        if sequence[i] == sequence_alignment[j] and sequence_alignment[j] != '-':
            if first_position == -1:
                first_position = i
            last_position = i
            j += 1
        i += 1

    return first_position+1

# Apply the function to each row in the DataFrame
df['alignment_pos'] = df.apply(find_first_last_aligned_positions, axis=1)

df

,sequence_id,sequence,rev_comp,productive,v_call,d_call,j_call,sequence_alignment,germline_alignment,junction,...,cdr3,cdr1_aa,cdr2_aa,cdr3_aa,locus,clone_id,frequency,v_alignment_length,v_alignment_mutation,alignment_pos
0,Seq_2757,TCTTCGGGAGGCACCTTCAGTACCTATGCTATCAGCTGGGTGCGAC...,F,T,IGHV1-69,IGHD3-10*01,IGHJ5,CTTCGGGAGGCACCTTCAGTACCTATGCTATCAGCTGGGTGCGACA...,CTTCGGGAGGCACCTTCAGTACCTATGCTATCAGCTGGGTGCGACA...,TGTGCGAGAGATCGGGGTAATATGGCTCGGGGAGTCTCCTGG,...,GCGAGAGATCGGGGTAATATGGCTCGGGGAGTCTCC,GGTFSTYA,IIPIFGTA,ARDRGNMARGVS,IGH,1,0.000031,226,11,2
1,Seq_298,TCTTCGGGAGGCACCTTCAGTACCTATGCTATCAGCTGGGTGCGAC...,F,T,IGHV1-69,IGHD3-10*01,IGHJ5,CTTCGGGAGGCACCTTCAGTACCTATGCTATCAGCTGGGTGCGACA...,CTTCGGGAGGCACCTTCAGTACCTATGCTATCAGCTGGGTGCGACA...,TGTGCGAGAGATCGGGGTAATATGGCTCGGGGAGTCTCCTGG,...,GCGAGAGATCGGGGTAATATGGCTCGGGGAGTCTCC,GGTFSTYA,IIPIFGTA,ARDRGNMARGVS,IGH,1,0.000687,226,13,2
2,Seq_2842,GCTTCTGGAGGCACCTTCAGCAGCTATGCTATCAGCTGGGTGCGAC...,F,T,IGHV1-69,IGHD3-10*01,IGHJ3,GCTTCTGGAGGCACCTTCAGCAGCTATGCTATCAGCTGGGTGCGAC...,GCTTCTGGAGGCACCTTCAGCAGCTATGCTATCAGCTGGGTGCGAC...,TGTGCGAGAAGTGAGGTTATAATCCAAGGTTTTGATATCTGG,...,GCGAGAAGTGAGGTTATAATCCAAGGTTTTGATATC,GGTFSSYA,IIPIFGTA,ARSEVIIQGFDI,IGH,2,0.000023,225,0,1
3,Seq_8553,ACTTCTGGAGGCACCCTCAGCAGCACTTATGCGATCAGTTGGGTGC...,F,T,IGHV1-69,IGHD3-22*01,IGHJ1,CTTCTGGAGGCACCCTCAGCAGCACTTATGCGATCAGTTGGGTGCG...,CTTCTGGAGGCACCCTCAGCAGCACTTATGCGATCAGTTGGGTGCG...,TGTGCGAGATATCCTACGAGTAGGGGTTATGTTGGGGAATACTTCC...,...,GCGAGATATCCTACGAGTAGGGGTTATGTTGGGGAATACTTCCAGCAC,GGTLSSTYA,IIPMFDIA,ARYPTSRGYVGEYFQH,IGH,3,0.000008,257,52,2
4,Seq_383,ACTTCTGGAGGCACCCTCAGCAGCACTTATGCGATCAGTTGGGTGC...,F,T,IGHV1-69,IGHD3-22*01,IGHJ1,CTTCTGGAGGCACCCTCAGCAGCACTTATGCGATCAGTTGGGTGCG...,CTTCTGGAGGCACCCTCAGCAGCACTTATGCGATCAGTTGGGTGCG...,TGTGCGAGATATCCTACGAGTAGGGGTTATGTTGGGGAATACTTCC...,...,GCGAGATATCCTACGAGTAGGGGTTATGTTGGGGAATACTTCCAGCAC,GGTLSSTYA,IIPMFDIA,ARYPTSRGYVGEYFQH,IGH,3,0.000578,257,51,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,Seq_3894,GCTTCTGGAGGCACCTTCAGCAGCTATGCTATCAGCTGGGTGCGAC...,F,T,IGHV1-69,IGHD5-18*01,IGHJ6,GCTTCTGGAGGCACCTTCAGCAGCTATGCTATCAGCTGGGTGCGAC...,GCTTCTGGAGGCACCTTCAGCAGCTATGCTATCAGCTGGGTGCGAC...,TGTGCGAGACCCCACGGGGGGTACAGCTATGGTTACTCCGACTACT...,...,GCGAGACCCCACGGGGGGTACAGCTATGGTTACTCCGACTACTACT...,GGTFSSYA,IIPILGIA,ARPHGGYSYGYSDYYYGMDV,IGH,119,0.000008,225,0,1
176,Seq_3184,GCTTCTGGAGGCACCTTCAGCAGCTATGCTATCAGCTGGGTGCGAC...,F,T,IGHV1-69,IGHD2-2*01,IGHJ6,GCTTCTGGAGGCACCTTCAGCAGCTATGCTATCAGCTGGGTGCGAC...,GCTTCTGGAGGCACCTTCAGCAGCTATGCTATCAGCTGGGTGCGAC...,TGTGCGAGACTAGGGCCTTTGGGGGTAGTACCAGCCGCGACTGGCT...,...,GCGAGACTAGGGCCTTTGGGGGTAGTACCAGCCGCGACTGGCTACT...,GGTFSSYA,IIPIFGTA,ARLGPLGVVPAATGYYGMDV,IGH,120,0.000016,225,0,1
177,Seq_5757,GCTTCTGGAGGCACCTTCAGCAGCTATGCTATCAGCTGGGTGCGAC...,F,T,IGHV1-69,IGHD1-7*01,IGHJ4,GCTTCTGGAGGCACCTTCAGCAGCTATGCTATCAGCTGGGTGCGAC...,GCTTCTGGAGGCACCTTCAGCAGCTATGCTATCAGCTGGGTGCGAC...,TGTGTGGCTGGAACTACGTGG,...,GTGGCTGGAACTACG,GGTFSSYA,IIPIFGTA,VAGTT,IGH,121,0.000008,220,0,1
178,Seq_7113,GCTTCTGGAGGCACCTTCAGCAGCTATGCTATCAGCTGGGTGCGAC...,F,T,IGHV1-69,IGHD4-4*01,IGHJ4,GCTTCTGGAGGCACCTTCAGCAGCTATGCTATCAGCTGGGTGCGAC...,GCTTCTGGAGGCACCTTCAGCAGCTATGCTATCAGCTGGGTGCGAC...,TGTGCGACCGCAGGTAACCCTGTCCTCTGG,...,GCGACCGCAGGTAACCCTGTCCTC,GGTFSSYA,IIPIFGTA,ATAGNPVL,IGH,122,0.000008,223,0,1


In [71]:
print(df['sequence'].iloc[0])
print(df['sequence_alignment'].iloc[0])

TCTTCGGGAGGCACCTTCAGTACCTATGCTATCAGCTGGGTGCGACAGGCCCCTGGACAAGGGCTTGAGTGGATTGGAGGGATCATCCCTATCTTTGGTACAGCAAACTACGCACAGAAGTTCCAGGACAGAGTCACGATTACCGCGGACGAATCCACGACCACAGTCTACATGGAAATGACCAGCCTGAGATCTGACGACACGGCCGTGTATTACTGTGCGAGAGATCGGGGTAATATGGCTCGGGGAGTCTCCTGGGGCCAGGGAACTCTGGTCGCCGTCTCCTCA
CTTCGGGAGGCACCTTCAGTACCTATGCTATCAGCTGGGTGCGACAGGCCCCTGGACAAGGGCTTGAGTGGATTGGAGGGATCATCCCTATCTTTGGTACAGCAAACTACGCACAGAAGTTCCAGGACAGAGTCACGATTACCGCGGACGAATCCACGACCACAGTCTACATGGAAATGACCAGCCTGAGATCTGACGACACGGCCGTGTATTACTGTGCGAGAGATCGGGGTAATATGGCTCGGGGAGTCTCCTGGGGCCAGGGAACTCTGGTCGCCGTCTCCTCA


In [70]:
sequence = df['sequence'].iloc[2]
sequence_alignment = df['sequence_alignment'].iloc[2]
# Find the longest contiguous matching subsequence (LCS)
matcher = difflib.SequenceMatcher(None, sequence, sequence_alignment)
print(lcs.a)
lcs = matcher.find_longest_match(0, len(sequence), 0, len(sequence_alignment))
print(lcs.a)
# Print the result
print(f"Positions in 'sequence' aligned with 'sequence_alignment': {lcs.a + 1} {lcs.a + lcs.size - 1}")

0
0
Positions in 'sequence' aligned with 'sequence_alignment': 1 287


[1] 0.1351258
[1] "S1_add_d_gene.tsv"
[1] 0.1283988
[1] "S10_add_d_gene.tsv"
[1] 0.1230054
[1] "S11_add_d_gene.tsv"
[1] 0.3309881
[1] "S12_add_d_gene.tsv"
[1] 0.1072065
[1] "S13_add_d_gene.tsv"
[1] 0.1704155
[1] "S14_add_d_gene.tsv"
[1] 0.1015698
[1] "S15_add_d_gene.tsv"
[1] 0.08006341
[1] "S16_add_d_gene.tsv"
[1] 0.0893344
[1] "S17_add_d_gene.tsv"
[1] 0.08324094
[1] "S18_add_d_gene.tsv"
[1] 0.07130541
[1] "S19_add_d_gene.tsv"
[1] 0.3394774
[1] "S2_add_d_gene.tsv"
[1] 0.09756968
[1] "S20_add_d_gene.tsv"
[1] 0.05905136
[1] "S3_add_d_gene.tsv"
[1] 0.3179943
[1] "S4_add_d_gene.tsv"
[1] 0.1132562
[1] "S5_add_d_gene.tsv"
[1] 0.07391558
[1] "S6_add_d_gene.tsv"
[1] 0.1454048
[1] "S7_add_d_gene.tsv"
[1] 0.1366331
[1] "S8_add_d_gene.tsv"
[1] 0.0990255
[1] "S9_add_d_gene.tsv"



In [ ]:
DefineClones.py -d S2_add_d_gene.tsv --act set --model ham \
    --norm len --dist 0.3394774
DefineClones.py -d S3_add_d_gene.tsv --act set --model ham \
    --norm len --dist 0.05905136
DefineClones.py -d S4_add_d_gene.tsv --act set --model ham \
    --norm len --dist 0.3179943
DefineClones.py -d S5_add_d_gene.tsv --act set --model ham \
    --norm len --dist 0.1132562
DefineClones.py -d S6_add_d_gene.tsv --act set --model ham \
    --norm len --dist 0.07391558
DefineClones.py -d S7_add_d_gene.tsv --act set --model ham \
    --norm len --dist 0.1454048
DefineClones.py -d S9_add_d_gene.tsv --act set --model ham \
    --norm len --dist 0.0990255
DefineClones.py -d S10_add_d_gene.tsv --act set --model ham \
    --norm len --dist 0.1283988
DefineClones.py -d S11_add_d_gene.tsv --act set --model ham \
    --norm len --dist 0.1230054
DefineClones.py -d S12_add_d_gene.tsv --act set --model ham \
    --norm len --dist 0.3309881
DefineClones.py -d S13_add_d_gene.tsv --act set --model ham \
    --norm len --dist 0.1072065
DefineClones.py -d S14_add_d_gene.tsv --act set --model ham \
    --norm len --dist 0.1704155
DefineClones.py -d S15_add_d_gene.tsv --act set --model ham \
    --norm len --dist 0.1015698
DefineClones.py -d S16_add_d_gene.tsv --act set --model ham \
    --norm len --dist 0.08006341
DefineClones.py -d S17_add_d_gene.tsv --act set --model ham \
    --norm len --dist 0.0893344
DefineClones.py -d S18_add_d_gene.tsv --act set --model ham \
    --norm len --dist 0.08324094
DefineClones.py -d S19_add_d_gene.tsv --act set --model ham \
    --norm len --dist 0.07130541
DefineClones.py -d S20_add_d_gene.tsv --act set --model ham \
    --norm len --dist 0.09756968
DefineClones.py -d S1_add_d_gene.tsv --act set --model ham \
    --norm len --dist 0.1351258
DefineClones.py -d S8_add_d_gene.tsv --act set --model ham \
    --norm len --dist 0.1366331

In [ ]:
BuildTrees.py -d S8_add_d_gene_clone-pass.tsv --outname ex --log ex.log --collapse --igphyml --clean all --nproc 1